In [1]:
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import numpy as np
from datasets import load_dataset, concatenate_datasets

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.adapters import AdapterArguments, AdapterTrainer, AdapterConfigBase, AutoAdapterModel, setup_adapter_training
from transformers import BertTokenizer, BertModelWithHeads, AdapterConfig, EvalPrediction, TextClassificationPipeline
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

from torch.utils.data import DataLoader
import torch

from pdb import set_trace
import transformers.adapters.composition as ac

from transformers.adapters.heads import ClassificationHead

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

adapter_info = {'cola': {'load_adapter': 'lingaccept/cola@ukp', 'adapter_config': 'pfeiffer'},
                'sst2': {'load_adapter': 'sentiment/sst-2@ukp', 'adapter_config': 'pfeiffer'},
                'mrpc': {'load_adapter': 'sts/mrpc@ukp',        'adapter_config': 'pfeiffer' },
                'qqp' : {'load_adapter': 'sts/qqp@ukp',         'adapter_config': 'pfeiffer' },
               }

In [2]:
task_name_1 = 'mrpc'
task_name_2 = 'sst2'
model_name_or_path = 'bert-base-uncased'
pad_to_max_length = True
max_seq_length = 128
output_dir = f'./tmp/parallel_training'
load_adapter_1 = adapter_info[task_name_1]['load_adapter']
load_adapter_2 = adapter_info[task_name_2]['load_adapter']
adapter_config_1 = AdapterConfigBase.load(adapter_info[task_name_1]['adapter_config'])
adapter_config_2 = AdapterConfigBase.load(adapter_info[task_name_2]['adapter_config'])

set_seed(0)

In [3]:
raw_datasets_1 = load_dataset("glue", task_name_1)
raw_datasets_2 = load_dataset("glue", task_name_2)

In [4]:
def get_num_labels(task_name, raw_datasets):
    # Labels
    if task_name_1 is not None:
        is_regression = task_name == "stsb"
        if not is_regression:
            label_list = raw_datasets["train"].features["label"].names
            num_labels = len(label_list)
        else:
            num_labels = 1
    else:
        # Trying to have good defaults here, don't hesitate to tweak to your needs.
        is_regression = raw_datasets["train"].features["label"].dtype in ["float32", "float64"]
        if is_regression:
            num_labels = 1
        else:
            # A useful fast method:
            # https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.unique
            label_list = raw_datasets["train"].unique("label")
            label_list.sort()  # Let's sort it for determinism
            num_labels = len(label_list)
    return num_labels, is_regression

num_labels_1, is_regression_1 = get_num_labels(task_name_1, raw_datasets_1)
num_labels_2, is_regression_2 = get_num_labels(task_name_2, raw_datasets_2)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
)

def get_data(task_name, raw_datasets):
    sentence1_key, sentence2_key = task_to_keys[task_name]

    if pad_to_max_length:
        padding = "max_length"

    def preprocess_function(examples):    
        # Tokenize the texts
        args = (
            (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
        )
        result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)
    
        # Map labels to IDs (not necessary for GLUE tasks)
        # if label_to_id is not None and "label" in examples:
            # result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
        result["label"] = [(l if l != -1 else -1) for l in examples["label"]]
        return result
    
    raw_datasets = raw_datasets.map(
        preprocess_function,
        batched=True,
        desc="Running tokenizer on dataset",
    )

    return raw_datasets

In [6]:
dataset1 = get_data(task_name_1, raw_datasets_1)
dataset2 = get_data(task_name_2, raw_datasets_2)

train_dataset_1 = dataset1['train']
train_dataset_2 = dataset2['train']

eval_dataset_1 = dataset1['validation']
eval_dataset_2 = dataset2['validation']

In [7]:
def get_compute_metrics(task_name, is_regression):
    metric = evaluate.load("glue", task_name)
    
    def compute_metrics(p: EvalPrediction):
        preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
        preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)
        if task_name is not None:
            result = metric.compute(predictions=preds, references=p.label_ids)
            if len(result) > 1:
                result["combined_score"] = np.mean(list(result.values())).item()
            return result
        elif is_regression:
            return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
        else:
            return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

    return compute_metrics

compute_metrics_1 = get_compute_metrics(task_name_1, is_regression_1)
compute_metrics_2 = get_compute_metrics(task_name_2, is_regression_2)

In [8]:
model = AutoAdapterModel.from_pretrained(
    model_name_or_path,
    ignore_mismatched_sizes=False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertAdapterModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# We use the AutoAdapterModel class here for better adapter support.
model = AutoAdapterModel.from_pretrained(
    model_name_or_path,
    ignore_mismatched_sizes=False
)


adapter1 = model.load_adapter(load_adapter_1, with_head=False)
adapter2 = model.load_adapter(load_adapter_2, with_head=False)

model.active_adapters = ac.Parallel(adapter1, adapter2)

# model.set_active_adapters(adapter1)

# model.active_head = [ac.parse_heads_from_composition(adapter1)]

# model.train_adapter(list(model.config.adapters))

model.freeze_model(True)

model.register_custom_head(f'parallel_{task_name_1}_{task_name_2}', ClassificationHead)
model.add_custom_head(head_type=f"parallel_{task_name_1}_{task_name_2}", head_name=f"parallel_{task_name_1}_{task_name_2}")

# model.add_classification_head(
#         task_name_1,
#         num_labels=num_labels_1,
#         # id2label={i: v for i, v in enumerate(label_list)} if not is_regression else None,
#     )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertAdapterModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
print(model.adapter_summary())

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
mrpc                     bottleneck          894,528       0.817       1       0
sst-2                    bottleneck          894,528       0.817       1       0
--------------------------------------------------------------------------------
Full model                               109,482,240     100.000               1


In [11]:
model.active_head

'parallel_mrpc_sst2'

In [12]:
for k, v in model.named_parameters():
    if v.requires_grad:
        print(k)

heads.parallel_mrpc_sst2.1.weight
heads.parallel_mrpc_sst2.1.bias
heads.parallel_mrpc_sst2.4.weight
heads.parallel_mrpc_sst2.4.bias


In [13]:
training_args = TrainingArguments(
    output_dir = output_dir,
    learning_rate=1e-4,
    num_train_epochs=6,
    label_names=['labels'],
    save_strategy='epoch',
    evaluation_strategy='epoch',
    per_device_train_batch_size=16,
    overwrite_output_dir=True
)

trainer_class = Trainer

trainer_1 = trainer_class(
    model=model,
    args=training_args,
    train_dataset=train_dataset_1,
    eval_dataset=eval_dataset_1,
    compute_metrics=compute_metrics_1,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

trainer_2 = trainer_class(
    model=model,
    args=training_args,
    train_dataset=train_dataset_2,
    eval_dataset=eval_dataset_2,
    compute_metrics=compute_metrics_2,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

# Training
checkpoint = None
# if training_args.resume_from_checkpoint is not None:
#     checkpoint = training_args.resume_from_checkpoint
# elif last_checkpoint is not None:
#     checkpoint = last_checkpoint
train_result_1 = trainer_1.train(resume_from_checkpoint=checkpoint)
train_result_2 = trainer_2.train(resume_from_checkpoint=checkpoint)
# metrics = train_result.metrics
# max_train_samples = len(train_dataset_1)
# metrics["train_samples"] = len(train_dataset_1)

# trainer.save_model()  # Saves the tokenizer too for easy upload

# trainer.log_metrics("train", metrics)
# trainer.save_metrics("train", metrics)
# trainer.save_state()

The following columns in the training set don't have a corresponding argument in `BertAdapterModel.forward` and have been ignored: idx, sentence1, sentence2. If idx, sentence1, sentence2 are not expected by `BertAdapterModel.forward`,  you can safely ignore this message.
/home/jaehan/anaconda3/envs/pytorch2.0/lib/python3.11/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3668
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 690
  Number of trainable parameters = 592130
/home/jaehan/anaconda3/envs/pytorch2.0/lib/python3.11/site-packages/torch/nn/parallel/_functions.py

Epoch,Training Loss,Validation Loss,Accuracy,F1,Combined Score
1,No log,0.535113,0.764706,0.845659,0.805183
2,No log,0.536777,0.769608,0.849359,0.809483
3,No log,0.527366,0.774510,0.851133,0.812821
4,No log,0.521838,0.774510,0.850163,0.812336
5,0.407700,0.535951,0.772059,0.849271,0.810665
6,0.407700,0.537866,0.774510,0.851133,0.812821


The following columns in the evaluation set don't have a corresponding argument in `BertAdapterModel.forward` and have been ignored: idx, sentence1, sentence2. If idx, sentence1, sentence2 are not expected by `BertAdapterModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
Saving model checkpoint to ./tmp/parallel_training/checkpoint-115
Configuration saved in ./tmp/parallel_training/checkpoint-115/config.json
Configuration saved in ./tmp/parallel_training/checkpoint-115/generation_config.json
Model weights saved in ./tmp/parallel_training/checkpoint-115/pytorch_model.bin
tokenizer config file saved in ./tmp/parallel_training/checkpoint-115/tokenizer_config.json
Special tokens file saved in ./tmp/parallel_training/checkpoint-115/special_tokens_map.json
/home/jaehan/anaconda3/envs/pytorch2.0/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all inp

Epoch,Training Loss,Validation Loss,Accuracy
1,0.126100,0.245488,0.915138
2,0.132400,0.256456,0.920872
3,0.124400,0.248952,0.917431
4,0.128500,0.252505,0.922018
5,0.129400,0.252353,0.920872
6,0.122200,0.254769,0.918578


The following columns in the evaluation set don't have a corresponding argument in `BertAdapterModel.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `BertAdapterModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 16
Saving model checkpoint to ./tmp/parallel_training/checkpoint-2105
Configuration saved in ./tmp/parallel_training/checkpoint-2105/config.json
Configuration saved in ./tmp/parallel_training/checkpoint-2105/generation_config.json
Model weights saved in ./tmp/parallel_training/checkpoint-2105/pytorch_model.bin
tokenizer config file saved in ./tmp/parallel_training/checkpoint-2105/tokenizer_config.json
Special tokens file saved in ./tmp/parallel_training/checkpoint-2105/special_tokens_map.json
/home/jaehan/anaconda3/envs/pytorch2.0/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were sc

In [14]:
inputs = {
    'input_ids': torch.tensor(eval_dataset_1['input_ids']).to('cuda' if torch.cuda.is_available() else 'cpu'),
    'token_type_ids': torch.tensor(eval_dataset_1['token_type_ids']).to('cuda' if torch.cuda.is_available() else 'cpu'),
    'attention_mask': torch.tensor(eval_dataset_1['attention_mask']).to('cuda' if torch.cuda.is_available() else 'cpu')
}

# model = torch.nn.DataParallel(model).to(device)
model = model.to(device)
# Inference
model.eval()
with torch.no_grad():
    output = model(**inputs)
    logits = output.logits
    
# Get predictions
prediction1 = logits.cpu().numpy()

# Evaluate
compute_results = compute_metrics_1(EvalPrediction(predictions=prediction1, label_ids=eval_dataset_1["label"]))
print(compute_results)

{'accuracy': 0.7328431372549019, 'f1': 0.8288854003139718, 'combined_score': 0.7808642687844369}


In [15]:
inputs = {
    'input_ids': torch.tensor(eval_dataset_2['input_ids']).to('cuda' if torch.cuda.is_available() else 'cpu'),
    'token_type_ids': torch.tensor(eval_dataset_2['token_type_ids']).to('cuda' if torch.cuda.is_available() else 'cpu'),
    'attention_mask': torch.tensor(eval_dataset_2['attention_mask']).to('cuda' if torch.cuda.is_available() else 'cpu')
}

# model = torch.nn.DataParallel(model).to(device)
model = model.to(device)
# Inference
model.eval()
with torch.no_grad():
    output= model(**inputs)
    logits = output.logits
    
# Get predictions
prediction1 = logits.cpu().numpy()

# Evaluate
compute_results = compute_metrics_2(EvalPrediction(predictions=prediction1, label_ids=eval_dataset_2["label"]))
print(compute_results)

{'accuracy': 0.9185779816513762}
